In [1]:
import pandas as pd
from datetime import date

In [2]:
from broker_limit import broker_limit
config_path="config.env"
broker_obj=broker_limit(config_path)
# broker_obj.ssh_host
conn=broker_obj.make_db_connection()

C:\Users\Lenovo\anaconda3\envs\Bobtail\Lib\site-packages\paramiko\pkey.py:82: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 48.0.0.
  "cipher": algorithms.TripleDES,
C:\Users\Lenovo\anaconda3\envs\Bobtail\Lib\site-packages\paramiko\transport.py:253: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 48.0.0.
  "class": algorithms.TripleDES,


In [ ]:
# date_start='2025-07-01'
# date_end='2025-07-31'
debtor_id="15932bdd-f6d8-402a-ab65-d1e742b81437"
# debtor_id="f7498290-c18c-439d-a90c-b2a717b8b36b"
# debtor_id = urllib.parse.quote(debtor_id)
query = "select * from invoices i where debtor_id=%s and created_at>=current_date-455 and i.approved_date is not null"
cur = conn.cursor()
cur.execute(query, (debtor_id,))
results=cur.fetchall()
col_name=[i[0] for i in cur.description]
invoice_df=pd.DataFrame(results, columns=col_name)
invoice_df['approved_accounts_receivable_amount']=invoice_df['approved_accounts_receivable_amount']/100

query = "select * from debtors d where id=%s"
cur = conn.cursor()
cur.execute(query, (debtor_id,))
results=cur.fetchall()
col_name=[i[0] for i in cur.description]
debtors_df=pd.DataFrame(results, columns=col_name)
debtors_df['debtor_limit']=debtors_df['debtor_limit']/100
debtors_df['approved_total']=debtors_df['approved_total']/100

query = "select * from brokers b where debtor_id=%s"
cur = conn.cursor()
cur.execute(query, (debtor_id,))
results=cur.fetchall()
col_name=[i[0] for i in cur.description]
brokers_df=pd.DataFrame(results, columns=col_name)

query = "select * from clients c"
cur = conn.cursor()
cur.execute(query)
results=cur.fetchall()
col_name=[i[0] for i in cur.description]
clients_df=pd.DataFrame(results, columns=col_name)

In [ ]:
date_today=date.today()
date_last_year=date_today - pd.Timedelta(days=365)
start_date=(date_last_year + pd.Timedelta(days=(8-date_last_year.isoweekday())%7) + pd.Timedelta(days=7))-pd.Timedelta(days=1)
# start_date=invoice_df['approved_date'].min().date()
end_date=date_today

broker_level=broker_limit.generate_segment_level_data(start_date, end_date, debtors_df, brokers_df, invoice_df, step='weekly')
broker_level_current=broker_limit.generate_segment_level_data(start_date=None, end_date=end_date, debtors_df=debtors_df, brokers_df=brokers_df, invoice_df=invoice_df, step='current')
# broker_level_df=broker_level[broker_level['dtp'].isna()==False]
broker_level_df=pd.concat([broker_level, broker_level_current], ignore_index=True)
broker_level_df.head()

In [ ]:
pivot_table, df_t, pivot_table_client_conc=broker_limit.generate_report(broker_level_df, broker_profile_report=False, cohort=['all', 12],payment_trend_count=13, payment_trend_step='daily', debtors_df=debtors_df, brokers_df=brokers_df, invoice_df=invoice_df)

In [19]:
df_t

snapshot_date,2025-08-03,2025-08-10,2025-08-17,2025-08-24,2025-08-31,2025-09-07,2025-09-14,2025-09-21,2025-09-28,2025-10-05,2025-10-12,2025-10-19,2025-10-23
invoice_approved,800.00,751.00,753.00,705.00,694.00,646.00,728.00,778.00,783.00,782.00,762.00,793.00,501.00
invoice_approved_dollars,1094425.55,972773.92,928217.70,905777.17,987597.38,915305.41,961513.16,1071677.39,1027148.55,1059133.84,976848.14,1014999.68,633029.69
open_invoices_in_point,4116647.79,4280050.94,4227544.90,4195522.10,4064915.83,4043661.79,4047879.97,3755686.20,2994630.98,3721098.57,4511354.92,4438171.27,4353984.37
invoice_paid,716.00,586.00,727.00,718.00,799.00,720.00,766.00,1009.00,1304.00,261.00,154.00,814.00,493.00
invoice_paid_dollars,1011621.08,809370.77,980723.74,937799.97,1118203.65,936559.45,957294.98,1363871.16,1788203.77,332666.25,186591.79,1088183.33,703456.59


In [ ]:
# with pd.ExcelWriter('C://Users//Lenovo//Documents//Work//payment_trend_report//TQL_21_10.xlsx') as writer:
#     pivot_table.to_excel(writer, sheet_name='Sheet1', index=False)
#     df_t.to_excel(writer, sheet_name='Sheet2', index=False)
pivot_table.to_csv('C://Users//Lenovo//Documents//Work//payment_trend_report//TQL_24_10.csv')
df_t.to_csv('C://Users//Lenovo//Documents//Work//payment_trend_report//TQL_24_10_daily_payments.csv')
# pivot_table_client_conc.to_csv('C://Users//Lenovo//Documents//Work//payment_trend_report//SAGE_23_10_client_conc.csv')

In [ ]:
# pivot_table.to_csv('C://Users//Lenovo//Documents//Work//payment_trend_report//TQL_21_10.csv')
# df_t.to_csv('C://Users//Lenovo//Documents//Work//payment_trend_report//TQL_21_10_current_daily.csv')
# !pip install openpyxl

In [ ]:
# client concentration

In [ ]:
years_in_business=(date.today()-invoice_df.sort_values('created_at', ascending=True).iloc[0]['created_at'].date()).days / 365

debtor_level=invoice_df.groupby('debtor_id').agg(unique_clients=('client_id', 'nunique'), total_approved_invoices_dollars=('approved_accounts_receivable_amount', 'sum'), total_approved_invoices=('id', 'nunique')).reset_index()
debtor_level['invoice_amount_per_client']=debtor_level['total_approved_invoices_dollars'] / debtor_level['unique_clients']
debtor_level['years_in_business']=years_in_business

debtor_client_level=invoice_df.groupby(['debtor_id','client_id']).agg(client_approved_invoices=('approved_accounts_receivable_amount', 'sum')).reset_index().sort_values('client_approved_invoices', ascending=False)

abc=debtor_client_level.merge(debtor_level, on='debtor_id', how='left')
abc['client_invoices_cumsum']=abc['client_approved_invoices'].cumsum()
abc['share']=(abc['client_invoices_cumsum']/abc['total_approved_invoices_dollars']) *100
abc = abc.reset_index().rename(columns={'index': 'row_number'})
abc['row_number'] += 1
# abc[abc['share']>=25]['row_number'].min()
# abc['share']
abc

In [ ]:
invoice_df.groupby(['debtor_id','client_id']).agg(client_approved_invoices=('approved_accounts_receivable_amount', 'sum')).reset_index().sort_values('client_approved_invoices', ascending=False)

In [ ]:
invoice_df